ArgoCD
======

![](https://argo-cd.readthedocs.io/en/stable/assets/argocd-ui.gif)

Quelle: [ArgoCD](https://argo-cd.readthedocs.io/)

- - -

[ArgoCD](https://argo-cd.readthedocs.io/) CD ist ein deklaratives GitOps-Continuous-Delivery-Tool für Kubernetes.

- - - 

### Installation


In [ ]:
%%bash
kubectl create namespace argocd
kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml
kubectl patch svc argocd-server -n argocd -p '{"spec": {"type": "LoadBalancer"}}'

In [ ]:
%%bash
while ! kubectl -n argocd get secret argocd-initial-admin-secret >/dev/null 2>&1; do echo "."; sleep 2; done

echo ""
echo "ArgoCD UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n argocd service/argocd-server -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "User     : admin"
echo "Password : $(kubectl -n argocd get secret argocd-initial-admin-secret -o jsonpath="{.data.password}" | base64 -d; echo)"

- - -

Microservices - REST Deployen
----------------------------------------

Dazu müssen wir zuerst eine YAML Datei für ArgoCD erstellen und dann diese Kubernetes übergeben

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: ms-rest
  namespace: argocd
spec:
  destination:
    namespace: ms-rest
    server: https://kubernetes.default.svc
  project: default
  source:
    path: 3-0-0-deployment
    repoURL: https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates.git
    targetRevision: HEAD
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
    syncOptions:
      - CreateNamespace=true
%EOF%

while ! kubectl get namespace ms-rest >/dev/null 2>&1; do echo "."; sleep 2; done
kubectl get pods,services --namespace ms-rest

echo ""
echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

- - -

Microservices - Big Ball of Mud Deployen
-----------------------------------------------------------

Dazu müssen wir zuerst eine YAML Datei für ArgoCD erstellen und dann diese Kubernetes übergeben


In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: ms-bbum
  namespace: argocd
spec:
  destination:
    namespace: ms-bbum
    server: https://kubernetes.default.svc
  project: default
  source:
    path: 3-2-0-deployment
    repoURL: https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates.git
    targetRevision: HEAD
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
    syncOptions:
      - CreateNamespace=true
%EOF%

while ! kubectl get namespace ms-bbum >/dev/null 2>&1; do echo "."; sleep 2; done
kubectl get pods,services --namespace ms-rest

echo ""
echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-bbum webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

- - -

### Aufräumen

Dazu sind die ArgoCD Ressourcen "Application" zu löschen und die erstellen Kubernetes Namespaces.

Und zum Schluss ArgoCD selber.


In [ ]:
! kubectl delete --namespace argocd application/ms-rest
! kubectl delete namespace ms-rest
! kubectl delete --namespace argocd application/ms-bbum
! kubectl delete namespace ms-bbum
! kubectl delete namespace argocd

- - -
### Quellen

* [Argo CD](https://argo-cd.readthedocs.io/en/stable/)
* Sourcecode: https://gitlab.com/ch-mc-b/autoshop-ms/app
* Kubernetes Deklarationen: https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates
* Container Registry: https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/container_registry